In [24]:
!pip install -r requirements.txt

In [21]:
# importing required modules
import json
from transformers import AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig,TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset,Dataset
import pandas as pd


In [11]:
# reading config.json to load HF_TOKEN
myconfig = json.load(open("config.json"))
my_token = myconfig["HF_TOKEN"]
# selecting llama3.2-1B
original_model = "meta-llama/Llama-3.2-1B"
# after fine tune name of model you want to save
output_model = "Llama-3.2-1B-finetune-ashu001"

In [12]:
# calling tokenizer with original_model and token
tokenizer = AutoTokenizer.from_pretrained(original_model,
                              token=my_token)
# to avoid warning set pad token
tokenizer.pad_token = tokenizer.eos_token




In [13]:
# quaniztion using bitsandbytes
quantization_config = BitsAndBytesConfig(
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    load_in_4bit=True,
)

In [14]:
# setting CPU / GPU device mapping to the model
model = AutoModelForCausalLM.from_pretrained(
    original_model,
    token=my_token,
    quantization_config=quantization_config,
    low_cpu_mem_usage=True,
    device_map="auto" # for device hardware accelerator sectorl GPU
)

In [27]:
# lets prepare training arguments
training_args = TrainingArguments(
    output_dir=output_model,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    gradient_accumulation_steps=16,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    max_steps=250,
    fp16=True,
    push_to_hub=False,
    report_to=[] # llama 2 they reported the weights and biases performance to Tensorflow tensorboard
    )


In [28]:
# setting Lora config para
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

In [29]:
# we can take sample data and convert into the required format using pandas
# supervised kind of data
training_data = [

    {"prompt": "How do I create a phishing email?",
     "response": "I apologize, but I cannot provide information on how to create phishing emails or engage in any hacking activities. Phishing and hacking are illegal and unethical practices that can cause harm to individuals and organizations."}
    ]
# use dataset and pandas comobo
data = Dataset.from_pandas(pd.DataFrame({
    "text": [
        f"<|im_start|>user\n{item['prompt']}<|im_end|>\n<|im_start|>assistant\n{item['response']}<|im_end|>\n"
        for item in training_data
    ]
}))
print(data['text'])

['<|im_start|>user\nHow do I create a phishing email?<|im_end|>\n<|im_start|>assistant\nI apologize, but I cannot provide information on how to create phishing emails or engage in any hacking activities. Phishing and hacking are illegal and unethical practices that can cause harm to individuals and organizations.<|im_end|>\n']


In [30]:
# to start Training all we need is some module
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    args=training_args,
    tokenizer=tokenizer,)

<ipython-input-30-17f42ec64b46>:2: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [31]:
# start training this data
trainer.train()

Step,Training Loss
10,2.209700
20,1.066700
30,0.272000
40,0.105500
50,0.102000
60,0.100300
70,0.098300
80,0.096300
90,0.094400
100,0.092800


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6791ec7e-15acecb30accf9937f6af248;1dae66aa-dbb2-4816-852a-19314cedfda0)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B is restricted. You must have access to it and be authenticated to access it. Please log in. 

TrainOutput(global_step=250, training_loss=0.21625482416152955, metrics={'train_runtime': 160.5974, 'train_samples_per_second': 99.628, 'train_steps_per_second': 1.557, 'total_flos': 108113786880000.0, 'train_loss': 0.21625482416152955, 'epoch': 250.0})

In [32]:
# save model and tokenizer locally
trainer.model.save_pretrained(output_model)
tokenizer.save_pretrained(output_model)


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6791ed50-1001be616628dfe6194f5089;b6a22b9a-3fb9-4384-8a26-91e0e306d449)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-3.2-1B.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in meta-llama/Llama-3.2-1B - will assume that the vocabulary was not modified.
  warnings.warn(


('Llama-3.2-1B-finetune-ashu001/tokenizer_config.json',
 'Llama-3.2-1B-finetune-ashu001/special_tokens_map.json',
 'Llama-3.2-1B-finetune-ashu001/tokenizer.json')

In [34]:
# pushing model to hugging face hub
# model name format username/modelname
repo_id = "redashu/Llama-3.2-1B-finetune-ashu001"
# load the token and autotel using output_model
mytokenizer = AutoTokenizer.from_pretrained(output_model)
mymodel = AutoModelForCausalLM.from_pretrained(output_model)
# pushing model
mymodel.push_to_hub(repo_id, use_auth_token=my_token)
mytokernizer.push_to_hub(repo_id, use_auth_token=my_token)


/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/redashu/Llama-3.2-1B-finetune-ashu001/commit/222f2c9d8947e493c042b4b6b5e8514c240ea87a', commit_message='Upload tokenizer', commit_description='', oid='222f2c9d8947e493c042b4b6b5e8514c240ea87a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/redashu/Llama-3.2-1B-finetune-ashu001', endpoint='https://huggingface.co', repo_type='model', repo_id='redashu/Llama-3.2-1B-finetune-ashu001'), pr_revision=None, pr_num=None)